In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, State

# Load and prepare data
gdp_df = pd.read_csv("data/Countries GDP 1960-2020.csv")
olympics_df = pd.read_csv("data/olympics_cleaned.csv")

# Melt GDP into long format
gdp_long = gdp_df.melt(id_vars=["Country Name", "Country Code"],
                       var_name="Year", value_name="GDP")
gdp_long["Year"] = pd.to_numeric(gdp_long["Year"])

# Standardize country names for Olympic data
country_name_mapping = {
    "USA": "United States",
    "UK": "United Kingdom",
    "South Korea": "Korea, Rep.",
}
olympics_df["country"] = olympics_df["country"].replace(country_name_mapping)

# Average GDP per country
avg_gdp = gdp_long.groupby("Country Name")["GDP"].mean().reset_index()
avg_gdp.columns = ["Country", "Average GDP"]

# Olympic host years
olympic_hosts = olympics_df[["country", "year"]].drop_duplicates()
olympic_hosts = olympic_hosts[olympic_hosts['year'] >= 1960]

# Determine custom range for color scale
quantile = avg_gdp["Average GDP"].quantile(0.8)
color_range_max = quantile

# Dash App
app = Dash(__name__)

app.layout = html.Div([
    html.H1("World GDP Map with Olympic Hosting Highlights"),
    dcc.Graph(id='gdp-map', style={'height': '600px'}),
    html.Div(id='selected-country', style={'display': 'none'}),
    dcc.Graph(id='gdp-line')
])

@app.callback(
    Output('gdp-map', 'figure'),
    Input('selected-country', 'children')
)
def update_map(_):
    fig = px.choropleth(avg_gdp,
                        locations="Country",
                        locationmode="country names",
                        color="Average GDP",
                        hover_name="Country",
                        color_continuous_scale="Viridis",
                        range_color=(0, color_range_max),
                        title="Average GDP by Country (1960–2020)")
    return fig

@app.callback(
    Output('selected-country', 'children'),
    Input('gdp-map', 'clickData')
)
def store_click_data(clickData):
    if clickData:
        return clickData['points'][0]['location']
    return "United States"

@app.callback(
    Output('gdp-line', 'figure'),
    Input('selected-country', 'children')
)
def update_line_chart(country):
    country_gdp = gdp_long[gdp_long["Country Name"] == country]
    host_years = olympic_hosts[olympic_hosts["country"] == country]["year"].values

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=country_gdp["Year"],
                             y=country_gdp["GDP"],
                             mode='lines+markers',
                             name='GDP'))

    for year in host_years:
        fig.add_vline(x=year, line=dict(color='red', dash='dash'),
                      annotation_text=f"{year}", annotation_position="top left")

    fig.update_layout(title=f"GDP Over Time: {country}",
                      xaxis_title="Year", yaxis_title="GDP",
                      height=500)
    return fig

if __name__ == '__main__':
    app.run(debug=True)
